# Análise dos dados abertos da ANTT

Os dados utilizados nesse notebook vêm da página de Dados Abertos da ANTT (Agência Nacional de Transportes Terrestres), mais especificamente do grupo "Rodovias", disponível no [link](https://dados.antt.gov.br/group/rodovias]). Para saber mais sobre o Portal de Dados Abertos da ANTT veja o seguinte [link](https://dados.antt.gov.br/about).

## Objetivo

O principal objetivo desse notebook é criar vizualizações para verificar a consistência dos dados fornecidos à ANTT pelas concessionárias de cada trecho de rodovia. Por exemplo, verficar os traçados das rodovias.

# Análises e Vizualizações


## Observações sobre o notebook

Este notebook está no [GitHub](https://github.com/GuiBatalhoti/Dados_ANTT) e está sendo desenvolvido de forma local, porém devido à necessidade de acesso de outros colaboradores, o notebook também está no Google Colab.

Na necessidade de execução pelo Colab, executar a seguinte célula:

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import folium
import shapely
# from shapely import wkt
# from shapely import frechet_distance

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Abrindo os dados de KM de Pista

Especificações em: data/km_pista/quilometro_principal_dicionario_dados.pdf

In [4]:
pista_principal_df = pd.read_csv('data/km_pista_principal/dados_dos_quilometro_principal.csv', sep=';', encoding='latin1')
pista_marginal_df = pd.read_csv('data/km_pista_marginal/dados_dos_quilometro_marginal.csv', sep=';', encoding='latin1')
sinalizacao_ultrapassagem_df = pd.read_csv('data/sinalizacao/proibido_ultrapassar.csv', sep=';', encoding='latin1')
sinalizacao_velocidade_df = pd.read_csv('data/sinalizacao/velocidade_maxima.csv', sep=';', encoding='latin1')
radar_df = pd.read_csv('data/radar/dados_dos_radares.csv', sep=';', encoding='latin1')
edificacao_operac_df = pd.read_csv('data/edificacao_operacional/dados_da_edificacao_operacional.csv', sep=';', encoding='latin1')

In [5]:
print('pista_principal_df:\n\n', pista_principal_df.head(), pista_principal_df.info())
print("=====================================================================================================")
print('pista_marginal_df:\n\n', pista_marginal_df.head(), pista_marginal_df.info())
print("=====================================================================================================")
print('sinalizacao_ultrapassagem_df:\n\n', sinalizacao_ultrapassagem_df.head(), sinalizacao_ultrapassagem_df.info())
print("=====================================================================================================")
print('sinalizacao_velocidade_df:\n\n', sinalizacao_velocidade_df.head(), sinalizacao_velocidade_df.info())
print("=====================================================================================================")
print('radar_df:\n\n', radar_df.head(), radar_df.info())
print("=====================================================================================================")
print('edificacao_operac_df:\n\n', edificacao_operac_df.head(), edificacao_operac_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255127 entries, 0 to 255126
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   concessionaria  255127 non-null  object
 1   ano_do_pnv_snv  255127 non-null  int64 
 2   rodovia         255127 non-null  object
 3   km_m            255127 non-null  object
 4   sentido         255127 non-null  object
 5   latitude        255127 non-null  object
 6   longitude       255127 non-null  object
dtypes: int64(1), object(6)
memory usage: 13.6+ MB
pista_principal_df:

           concessionaria  ano_do_pnv_snv    rodovia    km_m      sentido  \
0  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22,600  Decrescente   
1  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22,400  Decrescente   
2  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22,200  Decrescente   
3  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22,000  Decrescente   
4  AUTOPISTA FERNÃO DIAS        

In [6]:
pista_principal_df['latitude'] = pista_principal_df['latitude'].str.replace(',', '.').astype(float)
pista_principal_df['longitude'] = pista_principal_df['longitude'].str.replace(',', '.').astype(float)
pista_principal_df['km_m'] = pista_principal_df['km_m'].str.replace(',', '.').astype(float)
pista_marginal_df['latitude'] = pista_marginal_df['latitude'].str.replace(',', '.').astype(float)
pista_marginal_df['longitude'] = pista_marginal_df['longitude'].str.replace(',', '.').astype(float)
pista_marginal_df['km_m'] = pista_marginal_df['km_m'].str.replace(',', '.').astype(float)
# sinalizacao_ultrapassagem_df['latitude'] = sinalizacao_ultrapassagem_df['latitude'].str.replace(',', '.').astype(float)
# sinalizacao_ultrapassagem_df['longitude'] = sinalizacao_ultrapassagem_df['longitude'].str.replace(',', '.').astype(float)
sinalizacao_velocidade_df['latitude'] = sinalizacao_velocidade_df['latitude'].str.replace(',', '.').astype(float)
sinalizacao_velocidade_df['longitude'] = sinalizacao_velocidade_df['longitude'].str.replace(',', '.').astype(float)
radar_df['situacao'] = radar_df['situacao'].map({'Ativo': 1, 'Inativo': 0})
radar_df['latitude'] = radar_df['latitude'].str.replace(',', '.').astype(float)
radar_df['longitude'] = radar_df['longitude'].str.replace(',', '.').astype(float)
edificacao_operac_df['latitude'] = edificacao_operac_df['latitude'].str.replace(',', '.').astype(float)
edificacao_operac_df['longitude'] = edificacao_operac_df['longitude'].str.replace(',', '.').astype(float)


Colocando os dados de forma "Crua" no mapa, desconsiderando o sentido

In [8]:
mapa = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

cores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'gray', 'black']
rodovia = pista_principal_df['rodovia'].iloc[0]
sentido = pista_principal_df['sentido'].iloc[0]
pontos_linha  = []
for i, linha in pista_principal_df.iterrows():
    if linha['rodovia'] != rodovia:
        mapa.add_child(folium.PolyLine(locations=pontos_linha, 
                                       color=cores[np.random.randint(0, len(cores))], 
                                       alpha=0.7,
                                       weight=3, 
                                       popup=folium.Popup(f"{rodovia}\n{linha['concessionaria']}")))
        rodovia = linha['rodovia']
        sentido = linha['sentido']
        pontos_linha = []
    pontos_linha.append([linha['latitude'], linha['longitude']])
mapa.save('output/mapa_rodovias_cru.html')

É possível obervar que algumas rodovias apresentam "riscos" que  cortam todo o seu trajeto, o que pode ser um erro de preenchimento dos dados. Exemplos:
- BR-163/MS (RIOSP)
- BR-163/MT (CRO)
- BR-163/MT (ECO050)
- BR-364/MT (CRO)
- BR-70/MT (CRO)
- BR-116/RJ (RODOVIA DO AÇO)

Em alguns casos, podemos observar que o "risco" atravessa todo o comprimenro da rodovia, que pode estar relacionado à desconsideração do sentido da rodovia. Exemplos:
- BR-116/SP (AUTOPISTA REGIS BITTENCOURT)
- BR-262/MG (CRO)
- BR-153/MG (CONCEBRA)

Transformando os dados em GeoDataFrame para melhor manipulação

In [7]:
def df2gdf_linestring(df: pd.DataFrame, ignora_sentido: bool) -> gpd.GeoDataFrame:
    df.sort_values(['rodovia', 'km_m', 'sentido'], inplace=True, ignore_index=True)
    df_return = {
        "rodovia": [],
        "sentido": [],
        "concessionaria": [],
        "km": [],
        "ano_do_pnv_snv": [],
        "geometry": []
    }
    pontos = []

    linha_anterior = df.iloc[0]
    for i, linha in df.iterrows():
        if linha['rodovia'] != linha_anterior['rodovia'] or (linha['sentido'] != linha_anterior['sentido'] and not ignora_sentido):
            df_return["rodovia"].append(linha_anterior['rodovia'])
            df_return["sentido"].append(linha_anterior['sentido'])
            df_return["concessionaria"].append(linha_anterior['concessionaria'])
            df_return["km"].append(df[(df['rodovia'] == linha_anterior['rodovia']) &
                                  (df['sentido'] == linha_anterior['sentido']) &
                                  (df['concessionaria'] == linha_anterior['concessionaria'])]['km_m'].max())
            df_return["ano_do_pnv_snv"].append(linha_anterior['ano_do_pnv_snv'])
            if len(pontos) > 1:
                df_return["geometry"].append(shapely.geometry.LineString(pontos))
            else:
                df_return["geometry"].append(shapely.geometry.Point(pontos[0]))
            pontos = []
            
        pontos.append((linha['longitude'], linha['latitude']))
        linha_anterior = linha


    return gpd.GeoDataFrame(df_return, crs="EPSG:4326")

In [8]:
pista_principal_gdf = df2gdf_linestring(pista_principal_df, True)
pista_marginal_gdf = df2gdf_linestring(pista_marginal_df, True)
sinalizacao_velocidade_gdf = gpd.GeoDataFrame(sinalizacao_velocidade_df, crs="EPSG:4326", geometry=gpd.points_from_xy(sinalizacao_velocidade_df['longitude'], sinalizacao_velocidade_df['latitude']))
radar_gdf = gpd.GeoDataFrame(radar_df, crs="EPSG:4326", geometry=gpd.points_from_xy(radar_df['longitude'], radar_df['latitude']))
edificacao_operac_gdf = gpd.GeoDataFrame(edificacao_operac_df, crs="EPSG:4326", geometry=gpd.points_from_xy(edificacao_operac_df['longitude'], edificacao_operac_df['latitude']))

In [9]:
print('pista_principal_df:\n\n', pista_principal_gdf.head(), pista_principal_gdf.info())
print("=====================================================================================================")
print('pista_marginal_df:\n\n', pista_marginal_gdf.head(), pista_marginal_gdf.info())
print("=====================================================================================================")
print('sinalizacao_velocidade_df:\n\n', sinalizacao_velocidade_gdf.head(), sinalizacao_velocidade_gdf.info())
print("=====================================================================================================")
print('radar_df:\n\n', radar_gdf.head(), radar_gdf.info())
print("=====================================================================================================")
print('edificacao_operac_df:\n\n', edificacao_operac_gdf.head(), edificacao_operac_gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   rodovia         99 non-null     object  
 1   sentido         99 non-null     object  
 2   concessionaria  99 non-null     object  
 3   km              99 non-null     float64 
 4   ano_do_pnv_snv  99 non-null     int64   
 5   geometry        99 non-null     geometry
dtypes: float64(1), geometry(1), int64(1), object(3)
memory usage: 4.8+ KB
pista_principal_df:

   rodovia      sentido    concessionaria       km  ano_do_pnv_snv  \
0  092/PR  Decrescente  LITORAL PIONEIRO  324.090            2019   
1  151/PR    Crescente  LITORAL PIONEIRO  318.022            2019   
2  239/PR  Decrescente  LITORAL PIONEIRO   12.359            2019   
3  407/PR    Crescente  LITORAL PIONEIRO   17.990            2019   
4  408/PR    Crescente  LITORAL PIONEIRO   21.999            2019   

          

## Colocando todos os dados em um mapa interativo

In [32]:
mapa = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

folium.GeoJson(pista_principal_gdf, 
               name="Pista Principal", 
               style_function=lambda x: {'color': 'blue', 'weight': 5, 'opacity': 0.6},
               popup=folium.GeoJsonPopup(fields=['rodovia', 'concessionaria'], labels=True),
               tooltip=folium.GeoJsonTooltip(fields=['rodovia', 'concessionaria'], labels=True),
               zoom_on_click=True).add_to(mapa)

folium.GeoJson(pista_marginal_gdf, 
               name='Pista Marginal', 
               style_function=lambda x: {'color': 'red', 'weight': 5, 'opacity': 0.6},
               popup=folium.GeoJsonPopup(fields=['rodovia', 'concessionaria'], labels=True),
               tooltip=folium.GeoJsonTooltip(fields=['rodovia', 'concessionaria'], labels=True),
               zoom_on_click=True).add_to(mapa)

folium.GeoJson(sinalizacao_velocidade_gdf, name='Sinalização de Velocidade', marker=folium.Marker(), show=False).add_to(mapa)

folium.GeoJson(radar_gdf, 
               name='Radar', 
               marker=folium.Marker(icon=folium.Icon(icon='camera', prefix='fa')), 
               style_function=lambda x: {'markerColor': "orange"},
               tooltip=folium.GeoJsonTooltip(fields=['rodovia', 'concessionaria'], labels=True)).add_to(mapa)

folium.GeoJson(edificacao_operac_gdf, 
               name='Edificação Operacional', 
               marker=folium.Marker(icon=folium.Icon(icon='building', prefix='fa')), 
               style_function=lambda x: {'markerColor': "lightgray"},
               tooltip=folium.GeoJsonTooltip(fields=['rodovia', 'concessionaria'], labels=True)).add_to(mapa)

folium.LayerControl().add_to(mapa)
mapa.save('output/mapa_rodovias_completo.html')

In [11]:
# brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")
# cores = list(mcolors.TABLEAU_COLORS.keys())

# fig = plt.figure(figsize=(20,20), dpi=100)
# ax = fig.add_subplot(111)
# ax.set_title("Malha Rodoviária do Brasil, análise qualitativa")

# brasil.plot(ax=ax, color='lightgray', edgecolor='black')
# for i, (index, linha) in enumerate(pista_gdf.iterrows()):
#     if linha['rodovia'].startswith('BR'):
#         pista_gdf.iloc[[i]].plot(ax=ax, color=cores[i % len(cores)], linewidth=2, alpha=0.7, label=f"{linha['rodovia']} - {linha['concessionaria']}")

# plt.legend(title="Rodovias - Concessionárias", loc="upper left")
# plt.show()

In [13]:
# brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")
# cores = list(mcolors.TABLEAU_COLORS.keys())

# fig = plt.figure(figsize=(20,20), dpi=100)
# ax = fig.add_subplot(111)
# ax.set_title("Malha Rodoviária do Brasil, análise qualitativa")

# brasil.plot(ax=ax, color='lightgray', edgecolor='black')
# for i, (index, linha) in enumerate(pista_gdf.iterrows()):
#     pista_gdf.iloc[[i]].plot(ax=ax, color=cores[i % len(cores)], linewidth=2, alpha=0.7, label=f"{linha['rodovia']} - {linha['concessionaria']}")

# plt.legend(title="Rodovias - Concessionárias", loc="upper left")
# plt.show()

In [ ]:
brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")
cores = list(mcolors.TABLEAU_COLORS.keys())

fig = plt.figure(figsize=(10,10))

gs = fig.add_gridspec(1, 2, wspace=0, width_ratios=[1, 1], hspace=0, height_ratios=[1])
(ax1, ax2) = gs.subplots(sharex=True, sharey=True)

ax1.set_title("Rodovias Principais")
ax2.set_title("Entruncamentos e Contornos")

brasil.plot(ax=ax1, color='lightgray', edgecolor='black')
brasil.plot(ax=ax2, color='lightgray', edgecolor='black')
for i, (index, linha) in enumerate(pista_gdf.iterrows()):
    cor = cores[i % len(cores)]
    if linha['rodovia'].startswith('BR'):
        pista_gdf.iloc[[i]].plot(ax=ax1, color=cor, linewidth=2, alpha=0.7)
    pista_gdf.iloc[[i]].plot(ax=ax2, color=cor, linewidth=2, alpha=0.7)

# plt.legend(title="Rodovias - Concessionárias", loc="upper left")
plt.savefig('output/rodovias.pdf')

In [ ]:
pista_gdf['estado'] = pista_gdf['rodovia'].map(lambda x: x.split('/')[1])
pista_gdf = pista_gdf[['rodovia', 'estado', 'sentido', 'concessionaria', 'km', 'ano_do_pnv_snv', 'geometry']]
pista_gdf

comparação dos dados do OpenStreetMap com os dados da ANTT da BR-163

In [ ]:
df = pd.read_csv('data/rodovias_MT-MS/points.csv')
df['geometry'] = df.apply(wkt.loads)
pista_osm_gdf = gpd.GeoDataFrame(df, crs="")
pista_osm_gdf.set_crs(epsg=3857, inplace=True)
pista_osm_gdf.to_crs(epsg=4326, inplace=True)
#adicionar os pontos à uma lineString
pista_osm_line = geom.LineString(pista_osm_gdf['geometry'].values)
pista_osm_gdf = gpd.GeoDataFrame({'geometry': [pista_osm_line]}, crs="EPSG:4326")
pista_osm_gdf

Calculando a distância entre os pontos do OpenStreetMap e da ANTT

In [ ]:
rodovias_mt_ms = pista_gdf[pista_gdf['estado'].isin(['MT', 'MS'])]
rodovias_mt_ms

In [16]:
line_rodo_mt_ms = rodovias_mt_ms.unary_union
distancia = frechet_distance(line_rodo_mt_ms, pista_osm_line)

In [ ]:
print(f"A distância de Fréchet entre as rodovias de MT/MS e a linha do OSM é de {distancia}.")

In [ ]:
brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.set_title("Rodovias Principais")

brasil.plot(ax=ax, color='lightgray', edgecolor='black')
pista_osm_gdf.plot(ax=ax, color='red', alpha=0.7, label="BR-163 - OSM")
pista_gdf[pista_gdf['estado'] == 'MT'].plot(ax=ax, color='blue', alpha=0.7, label="BR-163 - ANTT")
